In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tv = TfidfVectorizer(use_idf=True) 
 
# just send in all your docs here 
# tv_vectors=tv.fit_transform(x_train)
# from sklearn.feature_selection import SelectKBest,f_classif
# fvalue_Best = SelectKBest(f_classif, k=200)
# X = fvalue_Best.fit_transform(tv_vectors,y_train)

In [6]:
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# import torchtext
import torch
import torch.nn.functional as F

import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
from pytictoc import TicToc

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words()
t = TicToc()

[nltk_data] Downloading package punkt to /home/ishikaa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ishikaa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
X,y = df['review'].values,df['sentiment'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y)

In [8]:
def preprocess(data):
  '''
  Input Pandas series
  '''

  #To LOWERCASE
  text = [ex.lower() for ex in data]

  #Remove HTML stuff
  text= [re.sub("(<.*?>)","",ex) for ex in text]  
  #remove non-ascii and digits
  text= [re.sub("(\\W|\\d)"," ",ex) for ex in text]  
    
  #Remove whitespace
  text= [ex.strip() for ex in text]
  #Remove multiple spaces
  text = [re.sub(' +', ' ',ex) for ex in text]

  #Remove stop words
  text = list(map(lambda words: ' '.join(word for word in words.split() if word not in stop),text))

  return text

In [22]:
t.tic()
x_train = preprocess(X_train)
x_test = preprocess(X_test)
t.toc()

Elapsed time is 479.935432 seconds.


In [75]:
# x_train_pre = x_train
# x_test_pre = x_test

In [49]:
y_train = np.asarray([0 if y=='positive' else 1 for y in y_train])
y_test = np.asarray([0 if y=='positive' else 1 for y in y_test])

## Some random shiz

In [77]:
X_train = x_train_pre
X_test = x_test_pre

In [79]:
from collections import Counter

In [80]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(X_train):
    # The sentences will be stored as a list of words/tokens
    X_train[i] = []
    # print(type(sentence))
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word])
        X_train[i].append(word)
    if i%5000 == 0:
      print(str(i) + "done")

print("100% done")

0done
5000done
10000done
15000done
20000done
25000done
30000done
35000done
100% done


In [81]:
words = {k:v for k,v in words.items() if v>100}
words = sorted(words,key=words.get,reverse=True)

In [82]:
words = ['_PAD','_UNK'] + words

In [83]:
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

In [84]:
for idx, sentence in enumerate(X_train):
  X_train[idx] = [word2idx[word] if word in word2idx else 0 for word in sentence]

In [87]:
def alter_length(sentences,length):
  new = np.zeros((len(sentences),length))

  for idx,sentence in enumerate(sentences):
    # print(idx)
    if len(sentence)!=0:
      new[idx,-len(sentence):] = np.array(sentence)[:length]
    
  return new

In [88]:
X_train = alter_length(X_train,200) 

In [89]:
X_train.shape

(37500, 200)

In [91]:
for idx, sentence in enumerate(X_test):
    X_test[idx] = [word2idx[word] if word in word2idx else 0 for word in sentence]

X_test = alter_length(X_test,200) 

In [93]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

In [94]:
batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [95]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [144]:
len(word2idx)+1

5393

In [164]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

In [170]:
from lstm import MyLSTM

In [171]:
lstm = MyLSTM(vocab=vocab_size,embed=embedding_dim,hidden=hidden_dim,layers=n_layers,out=output_size,                              device=device)